## Libraries and Loading

In [53]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
from collections import Counter

In [54]:
Complete_data = r"C:\Frank\UoT 2024-2025\MIE8888 Project\Data\Complete Data\21_treatements_backup_joined_patients_procedure_na_removed.csv"
Patient_Procedure = r"C:\Frank\UoT 2024-2025\MIE8888 Project\Data\Complete Data\21_treatements_backup_joined_patients_procedure_grouped.csv"
rule_2 = r"C:\Frank\UoT 2024-2025\MIE8888 Project\Data\Complete Data\rules_2.csv"

## Encoding Type Helper Function

In [55]:
# with open(Complete_data, 'rb') as f:
#     raw_data = f.read()
#     result = chardet.detect(raw_data)
#     encoding = result['encoding']
# print(f"Detected encoding: {encoding}")

In [56]:
# with open(Patient_Procedure, 'rb') as f:
#     raw_data = f.read()
#     result = chardet.detect(raw_data)
#     encoding2 = result['encoding']
# print(f"Detected encoding: {encoding}")

## Temporarily remove empty columns

In [57]:
df = pd.read_csv(Complete_data, encoding="Windows-1252")
cleaned_df = df.dropna(axis=1, how='all')
cleaned_column_names = cleaned_df.columns.tolist()
print(cleaned_column_names)
print(cleaned_df.shape[1])

original_column_names = df.columns.tolist()
dropped_columns = [col for col in original_column_names if col not in cleaned_column_names]
print(dropped_columns)
print(len(dropped_columns))

C:\Users\frank\AppData\Local\Temp\ipykernel_2248\2635263723.py:1: DtypeWarning: Columns (1,4,10,16,52,85,86,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Complete_data, encoding="Windows-1252")


['id_x_x', 'sikka_sub_task_x_x', 'sikka_sub_task_operation_x_x', 'name_x', 'office_x_x', 'patient', 'guarantor_x', 'provider_x', 'description', 'state_x', 'visible_to_patient', 'amount', 'estimated_cost', 'insurance_coverage', 'tooth_from', 'primary_insurance_estimate', 'secondary_insurance_estimate', 'plan_sr_no', 'entry_date', 'insurance_payment', 'treatment_plan_status', 'procedure_date', 'priority', 'approved', 'from_pms_x_x', 'id_y_x', 'from_pms_y_x', 'sikka_sub_task_y_x', 'sikka_sub_task_operation_y_x', 'uuid', 'patient_id', 'guarantor_y', 'office_y_x', 'fullname', 'first_name', 'middle_name', 'last_name', 'birth_date', 'address', 'city', 'state_y', 'first_visit', 'last_visit', 'email', 'phone', 'zip_code', 'practice', 'sikka_provider_id', 'primary_insurance_company_id', 'profile_picture', 'date_joined', 'zip_code_flag', 'address_comp', 'fetched_zip_code', 'fetched_zip_code_flag', 'zip_code_verified', 'final_zip_code', 'LATITUDE', 'LONGITUDE', 'id_x_y', 'from_pms_x_y', 'sikka_sub

## Task 1: Focus on the treatment processing per day, consider the ones with more than 7 treatment per day.

In [58]:
df2 = pd.read_csv(Patient_Procedure)
filtered_df = df2[df2['procedure_code_y_len'] >= 7]
unnormal_case = filtered_df[['procedure_code_y', 'procedure_code_y_len']]
unnormal_case.head()

,procedure_code_y,procedure_code_y_len
939,"[1203, 2142, 12101, 43412, 62501, 67211, 99111]",7
1458,"[1103, 1103, 2113, 2113, 2144, 2144, 2601, 260...",10
1475,"[1103, 2142, 2601, 11112, 11117, 12101, 99600]",7
2810,"[1103, 2112, 2142, 2601, 12101, 43412, 43413]",7
3156,"[1202, 2114, 11101, 11113, 12101, 25531, 27602...",9


In [59]:
case_counts = Counter(unnormal_case['procedure_code_y'])
case_summary = pd.DataFrame.from_dict(case_counts, orient='index', columns=['Count'])
case_summary.head()

,Count
"[1203, 2142, 12101, 43412, 62501, 67211, 99111]",1
"[1103, 1103, 2113, 2113, 2144, 2144, 2601, 2601, 23311, 23311]",1
"[1103, 2142, 2601, 11112, 11117, 12101, 99600]",1
"[1103, 2112, 2142, 2601, 12101, 43412, 43413]",1
"[1202, 2114, 11101, 11113, 12101, 25531, 27602, 99111, 99111]",1


In [60]:
# Consider the count of less than 10 as ramdom occurrence
case_summary = case_summary[case_summary['Count'] >= 10]
case_summary.head()

,Count
"[1202, 2144, 2144, 11107, 11107, 11113, 11113]",23
"[1103, 2112, 2144, 2601, 11103, 12101, 99700]",41
"[1202, 1202, 11107, 11107, 11113, 11113, 12111, 12111]",11
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113]",29
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113, 12111, 12111]",11


In [61]:
# Get the procedure code for special case and use them to find the treatment description in full dataset
unique_numbers = set()
for idx in case_summary.index:
    numbers = eval(idx) if isinstance(idx, str) else idx
    unique_numbers.update(numbers)
unique_numbers = sorted(unique_numbers)
print("Unique Codes", unique_numbers)


Unique Codes [1103, 1202, 2112, 2142, 2144, 2601, 11101, 11103, 11107, 11111, 11113, 11117, 12101, 12111, 96103, 99700]


In [62]:
special_descriptions = []
for i in unique_numbers:
    matching_rows = df[df['procedure_code_y'] == i]
    if not matching_rows.empty:
        special_descriptions.append({
            'procedure_code_y': i,
            'special_code_description': matching_rows['procedure_code_description'].iloc[0],  # Take the first match
            'count': len(matching_rows)
        })
summary = pd.DataFrame(special_descriptions).sort_values('count', ascending=False)
summary


,procedure_code_y,special_code_description,count
12,12101,!!!Fluoride Treatment Foam Gel or Rinse,25357
10,11113,Scaling Three Units,22911
1,1202,Exam & Diagnosis Limited Oral Previous Patient...,21516
6,11101,Polishing One Unit of Time,16154
11,11117,Scaling 1/2 Unit,15616
8,11107,Polishing 1/2 Unit of Time,15010
3,2142,Radiographs Intraoral Bitewing-2 Images,14629
4,2144,Radiographs Intraoral Bitewing - 4 Images,12256
5,2601,Radiographs Panoramic - 1 Image,11676
9,11111,Scaling One Unit of Time,7780


In [63]:
code_to_description = dict(zip(summary['procedure_code_y'], summary['special_code_description']))
case_summary['Description'] = case_summary.index.map(lambda codes: ", ".join([code_to_description[int(code)] for code in eval(codes) if int(code) in code_to_description]))
case_summary.sort_values('Count', ascending=False)

,Count,Description
"[1103, 2112, 2144, 2601, 11103, 12101, 99700]",41,Exam and Diagnosis Complete - Permanent Dentit...
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113]",29,Exam & Diagnosis Limited Oral Previous Patient...
"[1202, 2144, 2144, 11107, 11107, 11113, 11113]",23,Exam & Diagnosis Limited Oral Previous Patient...
"[1103, 2112, 2142, 2601, 11103, 12101, 99700]",20,Exam and Diagnosis Complete - Permanent Dentit...
"[1103, 2112, 2144, 2601, 11111, 96103, 99700]",16,Exam and Diagnosis Complete - Permanent Dentit...
"[1202, 2142, 2601, 11101, 11111, 11117, 12101]",14,Exam & Diagnosis Limited Oral Previous Patient...
"[1202, 1202, 11107, 11107, 11113, 11113, 12111, 12111]",11,Exam & Diagnosis Limited Oral Previous Patient...
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113, 12111, 12111]",11,Exam & Diagnosis Limited Oral Previous Patient...


- Above is one table that shows the **unnormal treatment groups** (more than 7 treatments applied to one patient one day) that have happened more than 10 times in our data.  
- This table includes the description of what the treatment group as well as the exact times these treatment group appeared.
- These groups may reflect specific orders of the treatment group that doctors already knew;  
- But some are possibly not, and we can use these groups for the use of next-step-treatment prediction reference

---

## Task 2 Use aprior algorithm on co-occurrence detection

In [64]:
# Load the data containing 
df_r = pd.read_csv(rule_2)

## Understand rule_2 csv
Support: 0.002914619: The rule applies to 0.29% of transactions.  
Confidence: 0.559360731: If item 62501 is purchased, there’s a 55.94% chance that 67211 and 99111 will also be purchased.  
Lift: 182.599: The consequent is 182.6 times more likely to occur given the antecedent, compared to random chance.  

- The 231 rows in data rule_2 has lift > 3 means the consequent is at least 3 times more likely to occur given the antecedent, compared to random chance.
- Confidence all greater than 50% means that if antecedent is purchased, higehr than 50% of the chance that the consequent will also be purchased.
- A high support value for an itemset indicates that the itemset occurs in a large proportion of the transactions.


## Define what High & Low

High support indicates that an itemset is frequently purchased, while high lift shows a strong association between items that occur together much more often than by chance  
Low support indicates that an itemset is less frequently purchased, while low lift shows a weak association between items that occur together much more often than by chance

- H_Support + H_Lift: Very popular treatment group with consequent frequently happened
- H_Support + L_Lift: Very popular treatment group with consequent less frequently happened
- L_Support + H_Lift: common treatment group with consequent frequently happened
- L_Support + L_Lift: common treatment group with consequent less frequently happened


In [65]:
support_threshold = 0.005
lift_threshold = 30.0

def classify_group(row):
    if row['support'] >= support_threshold and row['lift'] >= lift_threshold:
        return 'H_Support_H_Lift'  # Very popular with frequent consequent
    elif row['support'] >= support_threshold and row['lift'] < lift_threshold:
        return 'H_Support_L_Lift'  # Very popular with less frequent consequent
    elif row['support'] < support_threshold and row['lift'] >= lift_threshold:
        return 'L_Support_H_Lift'  # Common with frequent consequent
    else:
        return 'L_Support_L_Lift'  # Common with less frequent consequent

df_r['Group'] = df_r.apply(classify_group, axis=1)
df_r['Treatment_Group'] = df_r['antecedents'].str.findall(r'\d+').apply(lambda x: [int(i) for i in x])
df_r


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,Group,Treatment_Group
0,frozenset({62501}),"frozenset({67211, 99111})",0.005211,0.003063,0.002915,0.559361,182.599237,1.0,0.002899,2.262478,0.999733,0.543840,0.558007,0.755409,L_Support_H_Lift,[62501]
1,"frozenset({67211, 99111})",frozenset({62501}),0.003063,0.005211,0.002915,0.951456,182.599237,1.0,0.002899,20.492661,0.997579,0.543840,0.951202,0.755409,L_Support_H_Lift,"[67211, 99111]"
2,"frozenset({62501, 99111})",frozenset({67211}),0.003170,0.005074,0.002915,0.919325,181.189929,1.0,0.002899,12.332457,0.997644,0.546875,0.918913,0.746884,L_Support_H_Lift,"[62501, 99111]"
3,frozenset({67211}),"frozenset({62501, 99111})",0.005074,0.003170,0.002915,0.574443,181.189929,1.0,0.002899,2.342412,0.999552,0.546875,0.573090,0.746884,L_Support_H_Lift,[67211]
4,frozenset({62501}),frozenset({67211}),0.005211,0.005074,0.004735,0.908676,179.091158,1.0,0.004708,10.894442,0.999625,0.853162,0.908210,0.920926,L_Support_H_Lift,[62501]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,"frozenset({1202, 2114})",frozenset({11113}),0.017333,0.131342,0.008744,0.504461,3.840814,1.0,0.006467,1.752956,0.752685,0.062487,0.429535,0.285517,H_Support_L_Lift,"[1202, 2114]"
227,"frozenset({11117, 11101})",frozenset({12101}),0.018660,0.150757,0.010731,0.575072,3.814555,1.0,0.007918,1.998556,0.751876,0.067621,0.499639,0.323125,H_Support_L_Lift,"[11117, 11101]"
228,"frozenset({1202, 11117})",frozenset({12101}),0.022865,0.150757,0.011771,0.514828,3.414950,1.0,0.008324,1.750396,0.723718,0.072731,0.428701,0.296455,H_Support_L_Lift,"[1202, 11117]"
229,"frozenset({11113, 1204})",frozenset({12101}),0.009987,0.150757,0.005092,0.509827,3.381777,1.0,0.003586,1.732538,0.711402,0.032712,0.422812,0.271801,H_Support_L_Lift,"[11113, 1204]"


In [66]:
# H_Support + H_Lift
h_support_h_lift = df_r[df_r['Group'] == 'H_Support_H_Lift']
h_support_h_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,Group,Treatment_Group
7,frozenset({49510}),frozenset({49421}),0.009142,0.012015,0.007804,0.853611,71.043249,1.0,0.007694,6.749033,0.995021,0.584410,0.851831,0.751558,H_Support_H_Lift,[49510]
8,frozenset({49421}),frozenset({49510}),0.012015,0.009142,0.007804,0.649505,71.043249,1.0,0.007694,2.827023,0.997914,0.584410,0.646271,0.751558,H_Support_H_Lift,[49421]
14,frozenset({99700}),"frozenset({2112, 2144, 1103})",0.007661,0.013615,0.005252,0.685559,50.351599,1.0,0.005148,3.136946,0.987707,0.327765,0.681219,0.535658,H_Support_H_Lift,[99700]
18,frozenset({99700}),"frozenset({2144, 2601, 1103})",0.007661,0.015870,0.005383,0.702640,44.275259,1.0,0.005262,3.309555,0.984960,0.296624,0.697845,0.520923,H_Support_H_Lift,[99700]
19,"frozenset({2112, 99700})","frozenset({2144, 1103})",0.005978,0.019956,0.005252,0.878607,44.026720,1.0,0.005133,8.073311,0.983164,0.253955,0.876135,0.570898,H_Support_H_Lift,"[2112, 99700]"
20,"frozenset({99700, 1103})","frozenset({2112, 2144})",0.007519,0.016488,0.005252,0.698576,42.367674,1.0,0.005128,3.262884,0.983794,0.280051,0.693523,0.508559,H_Support_H_Lift,"[99700, 1103]"
21,frozenset({99700}),"frozenset({2112, 2144})",0.007661,0.016488,0.005300,0.691770,41.954913,1.0,0.005174,3.190839,0.983701,0.281161,0.686603,0.506599,H_Support_H_Lift,[99700]
22,"frozenset({2601, 99700})","frozenset({2144, 1103})",0.006561,0.019956,0.005383,0.820490,41.114476,1.0,0.005252,5.459537,0.982121,0.254714,0.816834,0.545118,H_Support_H_Lift,"[2601, 99700]"
27,frozenset({99700}),"frozenset({2144, 1103})",0.007661,0.019956,0.005924,0.773292,38.749416,1.0,0.005772,4.322933,0.981714,0.273101,0.768676,0.535081,H_Support_H_Lift,[99700]
28,"frozenset({2144, 99700})","frozenset({2112, 1103})",0.005984,0.022835,0.005252,0.877734,38.437827,1.0,0.005116,7.992096,0.979847,0.222867,0.874876,0.553871,H_Support_H_Lift,"[2144, 99700]"


In [67]:
# H_Support + L_Lift
h_support_L_lift = df_r[df_r['Group'] == 'H_Support_L_Lift']
h_support_L_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,Group,Treatment_Group
40,"frozenset({2144, 99700})","frozenset({2601, 1103})",0.005984,0.033340,0.005383,0.899602,26.982936,1.0,0.005184,9.628320,0.968736,0.158605,0.896140,0.530533,H_Support_L_Lift,"[2144, 99700]"
41,"frozenset({2112, 99700})","frozenset({2601, 1103})",0.005978,0.033340,0.005371,0.898507,26.950094,1.0,0.005172,9.524447,0.968685,0.158227,0.895007,0.529807,H_Support_L_Lift,"[2112, 99700]"
42,"frozenset({2112, 2601})","frozenset({2144, 1103})",0.021985,0.019956,0.011623,0.528680,26.491972,1.0,0.011184,2.079358,0.983883,0.383363,0.519082,0.555547,H_Support_L_Lift,"[2112, 2601]"
43,"frozenset({2144, 1103})","frozenset({2112, 2601})",0.019956,0.021985,0.011623,0.582414,26.491972,1.0,0.011184,2.342071,0.981847,0.383363,0.573028,0.555547,H_Support_L_Lift,"[2144, 1103]"
45,frozenset({99700}),"frozenset({2601, 1103})",0.007661,0.033340,0.006484,0.846273,25.383367,1.0,0.006228,6.288174,0.968020,0.187834,0.840971,0.520371,H_Support_L_Lift,[99700]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,"frozenset({1202, 2114})",frozenset({11113}),0.017333,0.131342,0.008744,0.504461,3.840814,1.0,0.006467,1.752956,0.752685,0.062487,0.429535,0.285517,H_Support_L_Lift,"[1202, 2114]"
227,"frozenset({11117, 11101})",frozenset({12101}),0.018660,0.150757,0.010731,0.575072,3.814555,1.0,0.007918,1.998556,0.751876,0.067621,0.499639,0.323125,H_Support_L_Lift,"[11117, 11101]"
228,"frozenset({1202, 11117})",frozenset({12101}),0.022865,0.150757,0.011771,0.514828,3.414950,1.0,0.008324,1.750396,0.723718,0.072731,0.428701,0.296455,H_Support_L_Lift,"[1202, 11117]"
229,"frozenset({11113, 1204})",frozenset({12101}),0.009987,0.150757,0.005092,0.509827,3.381777,1.0,0.003586,1.732538,0.711402,0.032712,0.422812,0.271801,H_Support_L_Lift,"[11113, 1204]"


In [68]:
# L_Support + H_Lift
L_support_h_lift = df_r[df_r['Group'] == 'L_Support_H_Lift']
L_support_h_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,Group,Treatment_Group
0,frozenset({62501}),"frozenset({67211, 99111})",0.005211,0.003063,0.002915,0.559361,182.599237,1.0,0.002899,2.262478,0.999733,0.543840,0.558007,0.755409,L_Support_H_Lift,[62501]
1,"frozenset({67211, 99111})",frozenset({62501}),0.003063,0.005211,0.002915,0.951456,182.599237,1.0,0.002899,20.492661,0.997579,0.543840,0.951202,0.755409,L_Support_H_Lift,"[67211, 99111]"
2,"frozenset({62501, 99111})",frozenset({67211}),0.003170,0.005074,0.002915,0.919325,181.189929,1.0,0.002899,12.332457,0.997644,0.546875,0.918913,0.746884,L_Support_H_Lift,"[62501, 99111]"
3,frozenset({67211}),"frozenset({62501, 99111})",0.005074,0.003170,0.002915,0.574443,181.189929,1.0,0.002899,2.342412,0.999552,0.546875,0.573090,0.746884,L_Support_H_Lift,[67211]
4,frozenset({62501}),frozenset({67211}),0.005211,0.005074,0.004735,0.908676,179.091158,1.0,0.004708,10.894442,0.999625,0.853162,0.908210,0.920926,L_Support_H_Lift,[62501]
5,frozenset({67211}),frozenset({62501}),0.005074,0.005211,0.004735,0.933177,179.091158,1.0,0.004708,14.886936,0.999487,0.853162,0.932827,0.920926,L_Support_H_Lift,[67211]
6,frozenset({49512}),frozenset({49421}),0.004098,0.012015,0.003545,0.865022,71.992936,1.0,0.003496,7.319585,0.990168,0.282063,0.863380,0.580036,L_Support_H_Lift,[49512]
9,frozenset({99700}),"frozenset({2112, 2601, 2144, 1103})",0.007661,0.011623,0.004824,0.629658,54.174467,1.0,0.004735,2.668826,0.989119,0.333608,0.625303,0.522352,L_Support_H_Lift,[99700]
10,"frozenset({2601, 99700})","frozenset({2112, 2144, 1103})",0.006561,0.013615,0.004824,0.735267,54.002487,1.0,0.004735,3.725966,0.987964,0.314219,0.731613,0.544785,L_Support_H_Lift,"[2601, 99700]"
11,"frozenset({99700, 1103})","frozenset({2112, 2601, 2144})",0.007519,0.012426,0.004824,0.641614,51.635639,1.0,0.004731,2.755615,0.988062,0.319040,0.637105,0.514919,L_Support_H_Lift,"[99700, 1103]"


In [69]:
# L_Support + L_Lift
L_support_L_lift = df_r[df_r['Group'] == 'L_Support_L_Lift']
L_support_L_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,Group,Treatment_Group
39,"frozenset({2112, 99700, 2144})","frozenset({2601, 1103})",0.005300,0.033340,0.004824,0.910213,27.301201,1.0,0.004647,10.766179,0.968505,0.142656,0.907117,0.527453,L_Support_L_Lift,"[2112, 99700, 2144]"
44,frozenset({14612}),frozenset({99111}),0.004360,0.031591,0.003533,0.810368,25.651950,1.0,0.003395,5.106790,0.965225,0.108991,0.804182,0.461106,L_Support_L_Lift,[14612]
48,"frozenset({2144, 2601, 99700, 1103})",frozenset({2112}),0.005383,0.035993,0.004824,0.896133,24.897706,1.0,0.004630,9.281135,0.965031,0.131977,0.892255,0.515080,L_Support_L_Lift,"[2144, 2601, 99700, 1103]"
49,"frozenset({2144, 2601, 99700})",frozenset({2112}),0.005437,0.035993,0.004866,0.894967,24.865327,1.0,0.004670,9.178154,0.965030,0.133073,0.891046,0.515076,L_Support_L_Lift,"[2144, 2601, 99700]"
54,"frozenset({2112, 2601, 99700, 2144})",frozenset({1103}),0.004866,0.044588,0.004824,0.991443,22.235771,1.0,0.004607,111.646748,0.959697,0.108090,0.991043,0.549817,L_Support_L_Lift,"[2112, 2601, 99700, 2144]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,"frozenset({1202, 11117, 2142})",frozenset({12101}),0.004759,0.150757,0.003111,0.653750,4.336443,1.0,0.002394,2.452687,0.773075,0.020412,0.592284,0.337193,L_Support_L_Lift,"[1202, 11117, 2142]"
218,"frozenset({11117, 11111})",frozenset({1202}),0.007792,0.126054,0.004080,0.523664,4.154274,1.0,0.003098,1.834726,0.765247,0.031445,0.454959,0.278017,L_Support_L_Lift,"[11117, 11111]"
220,"frozenset({2144, 11107})",frozenset({1202}),0.007007,0.126054,0.003575,0.510187,4.047356,1.0,0.002692,1.784243,0.758238,0.027608,0.439538,0.269273,L_Support_L_Lift,"[2144, 11107]"
222,"frozenset({2144, 11113, 1204})",frozenset({12101}),0.005782,0.150757,0.003414,0.590535,3.917126,1.0,0.002543,2.074030,0.749041,0.022297,0.517847,0.306591,L_Support_L_Lift,"[2144, 11113, 1204]"
